In [1]:
# This code sets up display options, imports, etc.
import sys
if sys.platform == 'emscripten':
    %pip install -q seaborn plotly
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

# set up plotting defaults
%matplotlib inline
set_matplotlib_formats("svg")
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (4, 3)
pio.templates.default = "simple_white"

# Sam's preferred styles
pio.templates["sam"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        width=600,
        height=400,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+sam"

# display options for numpy and pandas
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option("display.max_rows", 7)
pd.set_option("display.max_columns", 8)
pd.set_option("display.precision", 2)

# Lab 2: Exploratory Data Analysis

**Data Science Bootcamp with Python, Pandas, and Plotly**

## Dataset overview

### San Diego food safety

From [this article](https://inewsource.org/2023/02/09/san-diego-restaurants-food-safety-violations/) ([archive link](https://archive.ph/gz8BL)):

> In the last three years, one third of San Diego County restaurants have had at least one major food safety violation.

### 99% Of San Diego Restaurants Earn ‘A' Grades, Bringing Usefulness of System Into Question

From [this article](https://www.nbcsandiego.com/news/local/99-of-san-diego-restaurants-earn-a-grades-bringing-usefulness-of-system-into-question/25381/) ([archive link](https://archive.ph/yB6RU)):

> Food held at unsafe temperatures. Employees not washing their hands. Dirty countertops. Vermin in the kitchen. An expired restaurant permit.
> 
> Restaurant inspectors for San Diego County found these violations during a routine health inspection of a diner in La Mesa in November 2016. Despite the violations, the restaurant was awarded a score of 90 out of 100, the lowest possible score to achieve an ‘A’ grade.

### The data

- We downloaded the data about the 1000 restaurants closest to UC San Diego from [here](https://www.sandiegocounty.gov/content/sdc/deh/fhd/ffis/intro.html.html).
- We had to download the data as JSON files, then process it into DataFrames.

In [4]:
from pathlib import Path
rest_path = Path('data') / 'restaurants.csv'
insp_path = Path('data') / 'inspections.csv'
viol_path = Path('data') / 'violations.csv'

In [5]:
rest = pd.read_csv(rest_path)
insp = pd.read_csv(insp_path)
viol = pd.read_csv(viol_path)

<div class="alert alert-warning">
    <h3>Question 🤔</h3>
</div>
    
The first article said that one third of restaurants had at least one major safety violation.<br>
Which DataFrames and columns seem most useful to verify this?

In [7]:
rest.head(2)

In [8]:
rest.columns

In [9]:
insp.head(2)

In [10]:
insp.columns

In [11]:
viol.head(2)

In [12]:
viol.columns

### Introduction to `plotly`

### `plotly`

- We've used `plotly` last time briefly, but let's introduce it in more detail now.

- It's a visualization library that enables **interactive** visualizations.

<center><img src="imgs/plotly.png" width=40%></center>

### Using `plotly`

There are a few ways we can use `plotly`:

- Using the `plotly.express` syntax.
    - `plotly` is very flexible, but it can be verbose; `plotly.express` allows us to make plots quickly.
    - See the [**documentation here**](https://plotly.com/python/plotly-express) – it's very rich (there are good examples for almost everything).

- By setting `pandas` plotting backend to `'plotly'` (by default, it's `'matplotlib'`) and using the DataFrame `plot` method.

For now, we'll use `plotly.express` syntax:

In [43]:
import plotly.express as px

# Also, use plotly as default plotting engine
pd.options.plotting.backend = "plotly"

### Initial plots

First, let's look at the distribution of inspection `'score'`s:

In [44]:
fig = px.histogram(x=insp['score'])
fig

How about the distribution of average inspection `'score'` per `'grade'`?

In [45]:
scores = (
    insp[['grade', 'score']]
    .dropna()
    .groupby('grade')
    .mean()
    .reset_index()
)
# x= and y= are columns of scores. Convenient!
px.bar(scores, x='grade', y='score')

In [46]:
# Same as the above!
scores.plot(kind='bar', x='grade', y='score')

## Exploratory data analysis and feature types

### The data science lifecycle, revisited

<center>
    <img src="imgs/ds-lifecycle.svg" width=50%>
</center>

We're at the stage of **understanding the data**.

### Exploratory data analysis (EDA)

- Historically, data analysis was dominated by formal statistics, including tools like confidence intervals, hypothesis tests, and statistical modeling.

- In 1977, John Tukey [defined](https://search.worldcat.org/title/3058187) the term **exploratory data analysis**, which described a philosophy for proceeding about data analysis:

> Exploratory data analysis is actively incisive, rather than passively descriptive, with real emphasis on the discovery of the unexpected.

- Practically, EDA involves, among other things, computing summary statistics and drawing plots to understand the nature of the data at hand.

> The greatest gains from data come from surprises… The unexpected is best brought to our attention by **pictures**.

### Different feature types

<center><img src='imgs/data-types.png' width=90%></center>

<div class="alert alert-warning">
    <h3>Question 🤔</h3>
</div>
    
What is the **feature type** of each of the following variables?
    
- `insp['score']`
- `insp['grade']`
- `viol['violation_accela']`
- `viol['major_violation']`
- `rest['business_id']`
- `rest['opened_date']`

</div>



In [47]:
# Your code goes here.

### Feature types vs. data types

- The data type `pandas` uses is not the same as the "data type" we talked about just now!
    - There's a difference between feature type and computational data type.

- Take care when the two don't match up very well!

In [48]:
# pandas stores these as ints, but they're actually nominal.
rest['business_id']

In [49]:
# pandas stores these as strings, but they're actually numeric.
rest['opened_date']

## Data cleaning

### Four pillars of data cleaning

When loading in a dataset, to clean the data – that is, to prepare it for further analysis – we will:

1. Perform **data quality checks**.

2. Identify and handle **missing values**.

3. Perform **transformations**, including converting time series data to **timestamps**.

4. Modify **structure** as necessary.

## Data cleaning: Data quality checks

### Data quality checks

We often start an analysis by checking the quality of the data.

- Scope: Do the data match your understanding of the population? 
- Measurements and values: Are the values reasonable?
- Relationships: Are related features in agreement?
- Analysis: Which features might be useful in a future analysis? 

### Scope

Do the data match your understanding of the population?

We were told that we're only looking at the 1000 restaurants closest to UCSD, so the restaurants in `rest` should agree with that.

In [50]:
rest.sample(5)

### Measurements and values

Are the values reasonable?

Do the values in the `'grade'` column match what we'd expect grades to look like?

In [51]:
insp['grade'].value_counts()

What kinds of information does the `insp` DataFrame hold?

In [52]:
insp.info()

What's going on in the `'address'` column of `rest`?

In [53]:
# Are there multiple restaurants with the same address?
rest['address'].value_counts()

In [54]:
# Keeps all rows with duplicate addresses.
(
    rest
    .groupby('address')
    .filter(lambda df: df.shape[0] >= 2)
    .sort_values('address')
)

In [55]:
# Does the same thing as above!
(
    rest[rest.duplicated(subset=['address'], keep=False)]
    .sort_values('address')
)

### Relationships

Are related features in agreement?

Do the `'address'`es and `'zip'` codes in `rest` match?

In [56]:
rest[['address', 'zip']]

What about the `'score'`s and `'grade'`s in `insp`?

In [57]:
insp[['score', 'grade']]

### Analysis

Which features might be useful in a future analysis?

- We're most interested in:
    - These columns in the `rest` DataFrame: `'business_id'`, `'name'`, `'address'`, `'zip'`, and `'opened_date'`.
    - These columns in the `insp` DataFrame: `'business_id'`, `'inspection_id'`, `'score'`, `'grade'`, `'completed_date'`, and `'status'`.
    - These columns in the `viol` DataFrame: `'inspection_id'`, `'violation'`, `'major_violation'`, `'violation_text'`, and `'violation_accela'`.

- Also, let's rename a few columns to make them easier to work with.

### 💡 Pro-Tip: Using `pipe`

When we manipulate DataFrames, it's best to define individual functions for each step, then use the `pipe` **method** to chain them all together.

The `pipe` DataFrame method takes in a function, which itself takes in a DataFrame and returns a DataFrame.

- In practice, we would add functions one by one to the top of a notebook, then `pipe` them all.
- For today, will keep re-running `pipe` to show data cleaning process.

In [58]:
def subset_rest(rest):
    return rest[['business_id', 'name', 'address', 'zip', 'opened_date']]

rest = (
    pd.read_csv(rest_path)
    .pipe(subset_rest)
)
rest

In [59]:
# Same as the above – but the above makes it easier to chain more .pipe calls afterwards.
subset_rest(pd.read_csv(rest_path))

Let's use `pipe` to keep (and rename) the subset of the columns we care about in the other two DataFrames as well.

In [60]:
def subset_insp(insp):
    return (
        insp[['business_id', 'inspection_id', 'score', 'grade', 'completed_date', 'status']]
        .rename(columns={'completed_date': 'date'})
    )

insp = (
    pd.read_csv(insp_path)
    .pipe(subset_insp)
)

In [61]:
def subset_viol(viol):
    return (
        viol[['inspection_id', 'violation', 'major_violation', 'violation_accela']]
        .rename(columns={'violation': 'kind',
                         'major_violation': 'is_major',
                         'violation_accela': 'violation'})
    )

viol = (
    pd.read_csv(viol_path)
    .pipe(subset_viol)
)

### Combining the restaurant data

Let's join all three DataFrames together so that we have all the data in a single DataFrame.

In [62]:
def merge_all_restaurant_data():
    return (
        rest
        .merge(insp, on='business_id', how='left')
        .merge(viol, on='inspection_id', how='left')
    )

df = merge_all_restaurant_data()
df

<div class="alert alert-warning">
    <h3>Question 🤔</h3>
</div>
    
Why should the function above use two left joins? What would go wrong if we used other kinds of joins?



## Data cleaning: Missing values

### Missing values

Next, it's important to check for and handle missing values, as they can have a big effect on your analysis.

In [63]:
insp[['score', 'grade']]

In [64]:
# The proportion of values in each column that are missing.
insp.isna().mean()

In [65]:
# Why are there null values here?
# insp['inspection_id'] and viol['inspection_id'] don't have any null values...
df[df['inspection_id'].isna()]

There are many ways of handling missing values, which we'll cover in an entire lecture next week. But a good first step is to check how many there are!

## Data cleaning: Transformations and timestamps

### Transformations and timestamps

From last class:

> A transformation results from performing some operation on every element in a sequence, e.g. a Series.

It's often useful to look at ways of transforming your data to make it easier to work with.

- Type conversions (e.g. changing the string `"$2.99"` to the number `2.99`).

- Unit conversion (e.g. feet to meters).

- Extraction (Getting `'vermin'` out of `'Vermin Violation Recorded on 10/10/2023'`).

### Creating timestamps

Most commonly, we'll parse dates into `pd.Timestamp` objects.

In [66]:
# Look at the dtype!
insp['date']

In [67]:
# This magical string tells Python what format the date is in.
# For more info: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
date_format = '%Y-%m-%d'
pd.to_datetime(insp['date'], format=date_format)

In [68]:
# Another advantage of defining functions is that we can reuse this function
# for the 'opened_date' column in `rest` if we wanted to.
def parse_dates(insp, col):
    date_format = '%Y-%m-%d'
    dates = pd.to_datetime(insp[col], format=date_format)
    return insp.assign(**{col: dates})

insp = (
    pd.read_csv(insp_path)
    .pipe(subset_insp)
    .pipe(parse_dates, 'date')
)

# We should also remake df, since it depends on insp.
# Note that the new insp is used to create df!
df = merge_all_restaurant_data()

In [69]:
# Look at the dtype now!
df['date']

### Working with timestamps

- We often want to adjust granularity of timestamps to see overall trends, or seasonality.
- Use the `resample` method in `pandas` ([documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)).
    - Think of it like a version of `groupby`, but for timestamps.
    - For instance, `insp.resample('2W', on='date')` separates every two weeks of data into a different group.

In [70]:
insp.resample('2W', on='date')['score'].mean()

In [71]:
# Where are those numbers coming from?
insp[
    (insp['date'] >= pd.Timestamp('2020-01-05')) &
    (insp['date'] < pd.Timestamp('2020-01-19'))
]['score']

In [72]:
(insp.resample('2W', on='date')
 .size()
 .plot(title='Number of Inspections Over Time')
)

### The `.dt` accessor

Like with Series of strings, `pandas` has a `.dt` accessor for properties of timestamps ([documentation](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors)).

In [73]:
insp['date']

In [74]:
insp['date'].dt.day

In [75]:
insp['date'].dt.dayofweek

In [77]:
dow_counts = insp['date'].dt.dayofweek.value_counts()
fig = px.bar(dow_counts)
fig.update_xaxes(tickvals=np.arange(7), ticktext=['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])

## Data cleaning: Modifying structure

### Reshaping DataFrames

We often **reshape** the DataFrame's structure to make it more convenient for analysis. For example, we can:

- Simplify structure by removing columns or taking a set of rows for a particular period of time or geographic area.
    - We already did this!

- Adjust granularity by aggregating rows together.
    - To do this, use `groupby` (or `resample`, if working with timestamps).

- Reshape structure, most commonly by using the DataFrame `melt` method to un-pivot a dataframe.

### Using `melt`

- The `melt` method is common enough that we'll give it a special mention.
- We'll often encounter pivot tables (esp. from government data), which we call *wide* data.
- The methods we've introduced work better with *long-form* data, or *tidy* data.
- To go from wide to long, `melt`.

<center><img src='imgs/wide-vs-long.svg' width=40%></center>

### Example usage of `melt`

In [78]:
wide_example = pd.DataFrame({
    'Year': [2001, 2002],
    'Jan': [10, 130],
    'Feb': [20, 200],
    'Mar': [30, 340]
}).set_index('Year')
wide_example

In [79]:
wide_example.melt(ignore_index=False)

## Exploration

<div class="alert alert-warning">
    <h3>Question 🤔</h3>
</div>
    
What questions do you want me to try and answer with the data? I'll start with a single pre-prepared question, and then answer student questions until we run out of time.

### Example question: Can we rank restaurants by their number of violations? How about separately for each zip code?

And why would we want to do that? 🤔